In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)
data = pd.read_csv("jester-data-1.csv",header = None)
d = data.to_latex()
text_file = open("Output.txt_jokes", "w")
text_file.write(d)
text_file.close()

In [20]:
n_features = 2



train, validation, testing = np.split(data.sample(frac = 1), [int(.8*len(data)), int(.9*len(data))])
del train[0]
user_ratings = train.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

ValueError: cannot delete array elements

In [12]:
latent_item_features

array([[ 0.008,  0.301],
       [ 0.201,  0.834],
       [ 0.127,  0.374],
       [ 0.867,  0.982],
       [ 0.458,  0.016],
       [ 0.147,  0.415],
       [ 0.633,  0.162],
       [ 0.732,  0.092],
       [ 0.819,  0.097],
       [ 0.879,  0.616],
       [ 0.521,  0.883],
       [ 0.711,  0.27 ],
       [ 0.54 ,  0.069],
       [ 0.781,  0.914],
       [ 0.515,  0.853],
       [ 0.523,  0.135],
       [ 0.134,  0.204],
       [ 0.991,  0.883],
       [ 0.966,  0.274],
       [ 0.689,  0.433],
       [ 0.591,  0.27 ],
       [ 0.884,  0.303],
       [ 0.539,  0.934],
       [ 0.574,  0.329],
       [ 0.986,  0.589],
       [ 0.975,  0.68 ],
       [ 0.74 ,  0.947],
       [ 0.124,  0.066],
       [ 0.051,  0.433],
       [ 0.081,  0.352],
       [ 0.629,  0.499],
       [ 0.798,  0.798],
       [ 0.044,  0.417],
       [ 0.062,  0.285],
       [ 0.09 ,  0.759],
       [ 0.607,  0.037],
       [ 0.163,  0.147],
       [ 0.723,  0.806],
       [ 0.532,  0.868],
       [ 0.546,  0.691],


In [13]:
latent_user_preferences

array([[ 0.429,  0.425],
       [ 0.713,  0.078],
       [ 0.776,  0.09 ],
       ..., 
       [ 0.1  ,  0.843],
       [ 0.64 ,  0.444],
       [ 0.947,  0.399]])

In [14]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 30):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(rating!=99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%1 == 0 ):
            print mse
                    
                    
    


In [15]:
sgd()

26.2983595247
25.2793284492
24.9465773456
24.7017495829
24.467578751
24.2327214136
23.9960877781
23.7587564038
23.5224044124
23.2888667098
23.0599529894
22.837344228
22.6225250235
22.416739758
22.2209684411
22.035919558
21.8620370388
21.699518021
21.5483378932
21.4082792295
21.2789616095
21.1598698444
21.0503786856
20.9497726063
20.8572596622
20.771978748
20.6929997738
20.6193164413
20.5498314601
20.4833343269


In [16]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 1.061,  1.005,  0.971, ...,  1.257,  0.857,  1.462],
       [ 1.612,  1.741,  1.545, ...,  1.57 ,  0.911,  2.046],
       [ 1.458,  1.61 ,  1.409, ...,  1.364,  0.759,  1.822],
       ..., 
       [ 2.846,  2.446,  2.525, ...,  3.763,  2.751,  4.124],
       [ 2.74 ,  2.641,  2.524, ...,  3.172,  2.128,  3.738],
       [ 2.761,  2.822,  2.595, ...,  2.943,  1.853,  3.636]])

In [22]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
del data[0]
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,(99.000|1.061),(4.850|1.005),(-0.870|0.971),(0.240|0.419),(6.650|0.826),(4.510|1.635),(2.140|0.447),(4.810|0.106),(99.000|0.660),(3.690|1.194),...,(1.750|1.488),(99.000|1.311),(99.000|1.721),(99.000|1.156),(99.000|1.116),(99.000|1.371),(99.000|1.349),(99.000|1.257),(99.000|0.857),(99.000|1.462)
1,(7.770|1.612),(8.980|1.741),(7.430|1.545),(-8.400|0.859),(6.360|1.342),(-9.810|2.137),(-7.910|1.101),(3.450|0.436),(4.850|1.160),(9.170|1.735),...,(5.780|1.907),(9.030|1.626),(4.950|2.336),(-8.300|1.319),(7.280|1.413),(4.270|1.752),(3.690|1.756),(7.570|1.570),(3.110|0.911),(-9.220|2.046)
2,(99.000|1.458),(99.000|1.610),(99.000|1.409),(99.000|0.814),(1.600|1.228),(99.000|1.875),(-3.160|1.065),(-0.240|0.440),(99.000|1.075),(99.000|1.556),...,(99.000|1.666),(99.000|1.410),(99.000|2.066),(99.000|1.121),(99.000|1.231),(99.000|1.530),(99.000|1.540),(99.000|1.364),(99.000|0.759),(99.000|1.822)
3,(7.670|4.350),(7.910|4.024),(7.620|3.952),(7.770|1.617),(5.490|3.350),(8.160|6.860),(-6.210|1.647),(5.050|0.310),(99.000|2.638),(8.110|4.932),...,(99.000|6.260),(99.000|5.540),(99.000|7.181),(99.000|4.937),(99.000|4.703),(99.000|5.769),(99.000|5.663),(99.000|5.305),(99.000|3.689),(99.000|6.074)
4,(-9.420|1.435),(-1.700|1.371),(8.930|1.318),(99.000|0.579),(-1.460|1.122),(9.270|2.192),(-9.810|0.628),(-3.110|0.158),(99.000|0.902),(0.490|1.611),...,(99.000|1.993),(-9.710|1.753),(99.000|2.312),(99.000|1.540),(99.000|1.494),(99.000|1.836),(99.000|1.808),(99.000|1.681),(99.000|1.137),(99.000|1.968)
5,(99.000|3.441),(-3.500|3.164),(0.680|3.120),(4.370|1.260),(7.380|2.642),(6.410|5.454),(8.690|1.268),(0.730|0.223),(1.550|2.073),(8.690|3.907),...,(99.000|4.980),(99.000|4.412),(99.000|5.702),(99.000|3.941),(99.000|3.743),(99.000|4.590),(99.000|4.503),(99.000|4.223),(99.000|2.950),(99.000|4.818)
6,(99.000|0.907),(1.500|0.616),(99.000|0.752),(99.000|0.106),(2.180|0.609),(2.180|1.789),(2.330|-0.093),(2.620|-0.221),(99.000|0.387),(-1.800|1.110),...,(99.000|1.671),(99.000|1.534),(0.870|1.785),(99.000|1.482),(99.000|1.273),(99.000|1.545),(99.000|1.484),(99.000|1.457),(99.000|1.174),(99.000|1.447)
7,(3.160|-0.334),(-4.710|-0.646),(-4.220|-0.412),(99.000|-0.474),(5.340|-0.393),(-3.110|0.017),(-3.930|-0.787),(-6.550|-0.456),(-7.330|-0.449),(-6.800|-0.255),...,(99.000|0.074),(99.000|0.148),(99.000|-0.115),(99.000|0.306),(99.000|0.082),(99.000|0.075),(99.000|0.025),(99.000|0.124),(99.000|0.329),(99.000|-0.192)
8,(9.370|2.575),(8.500|2.583),(9.080|2.405),(9.080|1.166),(8.060|2.064),(-8.690|3.736),(7.040|1.369),(7.040|0.442),(8.640|1.708),(6.600|2.846),...,(9.080|3.374),(-8.540|2.937),(5.150|3.990),(3.400|2.513),(2.380|2.519),(-3.590|3.106),(0.630|3.078),(5.000|2.823),(5.000|1.818),(7.480|3.432)
9,(3.060|1.725),(-9.950|1.436),(-2.480|1.516),(1.070|0.476),(2.720|1.264),(0.000|2.978),(-0.440|0.341),(4.320|-0.082),(4.510|0.929),(-1.650|2.014),...,(99.000|2.745),(99.000|2.468),(99.000|3.054),(99.000|2.283),(99.000|2.075),(99.000|2.533),(99.000|2.464),(99.000|2.355),(7.180|1.753),(8.160|2.539)


In [23]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(99.0, 1.06096611315)","(4.85, 1.0046522545)","(-0.87, 0.971422479638)","(0.24, 0.418615349592)","(6.65, 0.826410245181)","(4.51, 1.63511620761)","(2.14, 0.447439597487)","(4.81, 0.105649159351)","(99.0, 0.660319754842)","(3.69, 1.19416007369)",...,"(1.75, 1.48806324495)","(99.0, 1.31123875974)","(99.0, 1.72081359237)","(99.0, 1.15647278696)","(99.0, 1.11609493842)","(99.0, 1.37087502874)","(99.0, 1.34921343526)","(99.0, 1.25681138065)","(99.0, 0.857079260744)","(99.0, 1.46220479202)"
1,"(7.77, 1.61197597148)","(8.98, 1.74142806884)","(7.43, 1.54528998111)","(-8.4, 0.859308907517)","(6.36, 1.34209402355)","(-9.81, 2.1366781631)","(-7.91, 1.10115740497)","(3.45, 0.43629529213)","(4.85, 1.16038906216)","(9.17, 1.73526582675)",...,"(5.78, 1.90665111306)","(9.03, 1.62607203004)","(4.95, 2.33584049775)","(-8.3, 1.31932953486)","(7.28, 1.4126429842)","(4.27, 1.75189882109)","(3.69, 1.75632895219)","(7.57, 1.57035275202)","(3.11, 0.910959058358)","(-9.22, 2.04621584463)"
2,"(99.0, 1.45779724114)","(99.0, 1.61040960868)","(99.0, 1.408955092)","(99.0, 0.814029210403)","(1.6, 1.22802525165)","(99.0, 1.87485000179)","(-3.16, 1.06548456417)","(-0.24, 0.440151850842)","(99.0, 1.0753771766)","(99.0, 1.5562232431)",...,"(99.0, 1.665732666)","(99.0, 1.41001919058)","(99.0, 2.06636218019)","(99.0, 1.12077988321)","(99.0, 1.23073401986)","(99.0, 1.52963285434)","(99.0, 1.53981187752)","(99.0, 1.36408819761)","(99.0, 0.759156862346)","(99.0, 1.82151319256)"
3,"(7.67, 4.35048831216)","(7.91, 4.0236532655)","(7.62, 3.95237154443)","(7.77, 1.61691938488)","(5.49, 3.35010826935)","(8.16, 6.85987248267)","(-6.21, 1.64675807893)","(5.05, 0.310185586205)","(99.0, 2.63753930112)","(8.11, 4.931927628)",...,"(99.0, 6.25982774573)","(99.0, 5.54007774562)","(99.0, 7.18051104517)","(99.0, 4.93740183343)","(99.0, 4.70283582071)","(99.0, 5.76890454724)","(99.0, 5.66342259822)","(99.0, 5.30486457764)","(99.0, 3.68898977847)","(99.0, 6.0740063804)"
4,"(-9.42, 1.43483492922)","(-1.7, 1.37058964861)","(8.93, 1.31758043121)","(99.0, 0.578500421967)","(-1.46, 1.12241699848)","(9.27, 2.19204656337)","(-9.81, 0.62845510705)","(-3.11, 0.158157805766)","(99.0, 0.901712704921)","(0.49, 1.61058316367)",...,"(99.0, 1.99280849177)","(-9.71, 1.75301316684)","(99.0, 2.31176307129)","(99.0, 1.5397429949)","(99.0, 1.4937055963)","(99.0, 1.83561566192)","(99.0, 1.80838975254)","(99.0, 1.68090091992)","(99.0, 1.13742461363)","(99.0, 1.96774722627)"
5,"(99.0, 3.4406318621)","(-3.5, 3.16449328274)","(0.68, 3.12008060142)","(4.37, 1.26042161982)","(7.38, 2.64236741892)","(6.41, 5.45375463916)","(8.69, 1.26775505604)","(0.73, 0.222666028799)","(1.55, 2.07302284775)","(8.69, 3.90696370876)",...,"(99.0, 4.97974483336)","(99.0, 4.41150158628)","(99.0, 5.70167213941)","(99.0, 3.94074861053)","(99.0, 3.742538222)","(99.0, 4.5895795405)","(99.0, 4.50309136768)","(99.0, 4.22326539506)","(99.0, 2.94960838476)","(99.0, 4.81809802879)"
6,"(99.0, 0.906606791344)","(1.5, 0.615878252406)","(99.0, 0.751826807259)","(99.0, 0.105767128742)","(2.18, 0.608586072424)","(2.18, 1.789471365)","(2.33, -0.0930756108459)","(2.62, -0.220893328579)","(99.0, 0.386949127831)","(-1.8, 1.10964958207)",...,"(99.0, 1.67126297062)","(99.0, 1.53361587063)","(0.87, 1.78489213133)","(99.0, 1.48215520589)","(99.0, 1.27314076625)","(99.0, 1.54483704583)","(99.0, 1.48419283438)","(99.0, 1.45667025694)","(99.0, 1.17383847765)","(99.0, 1.44735944826)"
7,"(3.16, -0.334318111101)","(-4.71, -0.646031359674)","(-4.22, -0.412383752399)","(99.0, -0.474045560021)","(5.34, -0.392934132066)","(-3.11, 0.0174296470121)","(-3.93, -0.786602198555)","(-6.55, -0.455857856975)","(-7.33, -0.448844120808)","(-6.8, -0.2551192041)",...,"(99.0, 0.0738839968234)","(99.0, 0.147869359374)","(99.0, -0.115237224152)","(99.0, 0.306346965807)","(99.0, 0.0820864710227)","(99.0, 0.0751129729853)","(99.0, 0.0247210727578)","(99.0, 0.123686988302)","(99.0, 0.329418455956)","(99.0, -0.192019357748)

In [24]:
d = comparison_data.to_latex()
text_file = open("comparison_jokes.txt", "w")
text_file.write(d)
text_file.close()